<a href="https://colab.research.google.com/github/das-apratim/GeospatialDeepLearning/blob/main/LULC_Classification_Using_MachineLearning_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Image Classification Using PCA & K-Means**  

Image classification using **Principal Component Analysis (PCA) and K-Means clustering** is an **unsupervised learning approach** for segmenting satellite imagery into distinct land cover classes.  

#### **Step 1: Principal Component Analysis (PCA)**
PCA reduces the **dimensionality** of multispectral satellite images while preserving the most significant spectral information. By transforming correlated bands into a new set of **principal components**, it helps:  
- Remove redundant information  
- Highlight variance in spectral data  
- Improve clustering performance  

Typically, the **top 2-3 principal components** are used for further classification.  

#### **Step 2: K-Means Clustering**  
After PCA transformation, **K-Means clustering** is applied to group pixels into different land cover categories. K-Means:  
- Assigns each pixel to the nearest cluster based on spectral similarity  
- Segments the image into distinct land cover classes (e.g., vegetation, water, urban areas)  
- Works without labeled training data (unsupervised classification)  

This method is widely used in remote sensing for **Land Use/Land Cover (LULC) classification**, especially when ground truth labels are unavailable.  

#### **Applications in Remote Sensing**  
- Urban expansion monitoring  
- Vegetation mapping (NDVI-based classification)  
- Water body identification  
- Disaster damage assessment  

This approach is useful for rapid land cover classification without requiring supervised training data.



## Download Sample Data

In [ ]:
!wget https://github.com/das-apratim/GeospatialDeepLearning/blob/main/data/sampled_data.zip?raw=true -O sampled_data.zip
!unzip -q sampled_data.zip -d nz_imagery_sample

## Install Required Libraries

In [ ]:
!pip install -q rasterio

## Import Libraries

In [ ]:
import numpy as np
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import os
from scipy.ndimage import generic_filter
from glob import glob
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

## Read Sentinal Bands

In [ ]:
# Sentinel-2 band files (Modify this for your dataset)
out_profile= None
transform = None
band_files = glob("nz_imagery_sample/*.tif")

h = 0
w = 0
bands = {}

with rasterio.open(band_files[0]) as data:
    out_profile = data.profile.copy()
    transform = data.transform
    crs = data.crs
    h = data.height
    w = data.width

out_profile.update({"transform": transform})
out_profile.update({"crs": crs})

for f in band_files:
  data = rasterio.open(f)
  ras_data = data.read(1)
  ras_data = ras_data[0:h, 0:w]
  bands[f.split("/")[-1].split(".")[0]] = ras_data

## Canculate Supportive Indices (NDVI, NDBI, NDWI)
### **NDVI, NDBI, NDWI and Their Role in Image Classification**  

In remote sensing, spectral indices like **NDVI (Normalized Difference Vegetation Index), NDBI (Normalized Difference Built-up Index), and NDWI (Normalized Difference Water Index)** help enhance specific land cover features for classification. These indices are derived from multispectral satellite imagery and play a crucial role in distinguishing vegetation, urban areas, and water bodies.  

#### **1. NDVI (Normalized Difference Vegetation Index)**  
NDVI is used to assess vegetation health and distribution. It is calculated using the Near-Infrared (NIR) and Red bands:  
- Higher NDVI values indicate dense vegetation.  
- Lower NDVI values suggest barren land, urban areas, or water.  

#### **2. NDBI (Normalized Difference Built-up Index)**  
NDBI helps in identifying built-up and urbanized areas. It is derived from the Shortwave Infrared (SWIR) and Near-Infrared (NIR) bands:  
- High NDBI values indicate built-up regions.  
- Low values represent vegetation, water, or bare land.  

#### **3. NDWI (Normalized Difference Water Index)**  
NDWI is used for water body detection and is calculated using the Green and Near-Infrared (NIR) bands:  
- Higher NDWI values highlight water bodies.  
- Lower values indicate land features like vegetation or urban areas.  

#### **Role in Image Classification**  
These indices serve as additional input bands in classification models such as **PCA + K-Means, Random Forest, or SVM** by:  
- Enhancing spectral differences between land cover types.  
- Improving accuracy in distinguishing vegetation, water, and urban areas.  
- Reducing misclassification by incorporating meaningful spectral features.  

By integrating NDVI, NDBI, and NDWI with multispectral bands, classification results become more precise, aiding in better Land Use/Land Cover (LULC) mapping.

In [ ]:
## Add Derived Bands
epsilon = 1e-6

# Compute NDVI, NDWI, and NDBI
ndvi = (bands["B08_10m"] - bands["B04_10m"]) / (bands["B08_10m"] + bands["B04_10m"] + epsilon)
ndwi = (bands["B03_10m"] - bands["B08_10m"]) / (bands["B03_10m"] + bands["B08_10m"] + epsilon)
ndbi = (bands["resampled_B11_20m"] - bands["B08_10m"]) / (bands["resampled_B11_20m"] + bands["B08_10m"] + epsilon)


# Read bands and stack them
stacked_image = np.stack([bands["B02_10m"], bands["B03_10m"], bands["B04_10m"], bands["B08_10m"], bands["resampled_B11_20m"], bands["resampled_B12_20m"], ndvi, ndwi, ndbi], axis=-1)

## Compute PCA for The whole image with 3 Principal components

In [ ]:
# Get Image Shape
height, width, n_bands = stacked_image.shape
print(f"Original Image Shape: {stacked_image.shape}")

# Reshape to (n_samples, n_features) for PCA
pixels = stacked_image.reshape(-1, n_bands)  # Flatten to (H*W, Bands)

# Normalize Pixel Values
scaler = StandardScaler()
pixels_norm = scaler.fit_transform(pixels)

# Apply PCA - Keep only 3 principal components
n_components = 3
pca = PCA(n_components=n_components)
pca_result = pca.fit_transform(pixels_norm)

# Reshape back to Image Shape (Height, Width, Components)
pca_image = pca_result.reshape(height, width, n_components)

print(f"PCA Image Shape: {pca_image.shape}")

# Explained Variance
print("Explained Variance per Component:", pca.explained_variance_ratio_)
print("Total Variance Captured:", sum(pca.explained_variance_ratio_))

### Save PCA Components as MultiBand Raster and Preview Image

In [ ]:
## Set Raster Profile
out_profile.update({"count": n_components})
out_profile.update({"transform": transform})

# Save PCA Components as a Multi-band GeoTIFF
with rasterio.open("pca_with_indices.tif","w",**out_profile) as dst:
    for i in range(n_components):
        dst.write(pca_image[:, :, i], i+1)

print("PCA-transformed image saved as 'pca_with_indices.tif'")

with rasterio.open("pca_with_indices.tif") as src:
    pca_data = src.read()
    show(pca_data, cmap='jet') # You can change the colormap
    plt.show()

## K-Means clustring

#### Setting Up K-Means for 5 Primary classes and Saving the output

In [ ]:
# Flatten PCA image for clustering
pca_pixels = pca_image.reshape(-1, n_components)  # Shape: (n_samples, 3)

# Apply K-Means
n_clusters = 5  # Modify as needed
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
clusters = kmeans.fit_predict(pca_pixels)

# Reshape back to image dimensions
clustered_map = clusters.reshape(height, width)

# Save K-Means Clustering Output
out_profile.update({"count": 1})

with rasterio.open("pca_kmeans_indices.tif","w",**out_profile) as dst:
    dst.write(clustered_map.astype(rasterio.uint8), 1)

print("Clustered image saved as 'pca_kmeans_indices.tif'")

## Preview The Classified Data

In [ ]:
with rasterio.open("pca_kmeans_indices.tif") as src:
    pca_data = src.read()
    show(pca_data) # You can change the colormap
    plt.show()